In [1]:
import sys
!{sys.executable} -m pip install PyAthena


    100% |████████████████████████████████| 829kB 28.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://aws-athena-query-results-984073016564-us-west-2/sagemaker', region_name='us-west-2')
df = pd.read_sql("SELECT distinct * FROM retsdata.union_geo2 WHERE city = 'San Diego'", conn)

In [3]:
len(df)

168786

In [4]:
df.sort_values(by=['zip5'])[:10]

,streetnumber,streetdirprefix,streetname,city,state,zip5,directions,cmplx,neighborhd
26199,2626,None,Coronado Ave,San Diego,CA,88888,"I-5 south, exit on Coronado Ave, left turn Cro...",South Sd,South Sd
168382,2626,None,Coronado Ave,San Diego,CA,88888,"I-5 south, exit on Coronado Ave, left turn",Helena Mobile Home Park,Helena Mobile Park
75687,1915,None,Ranchito Ln,San Diego,CA,91901,Cross Street: Alpine Heights.,Alpine,Alpine
116941,3033,None,Via Asoleado,San Diego,CA,91901,8E - Tavern Rd (go South) - S Grade (go East) ...,Rancho Palo Verde,Alpine
4530,3033,None,Via Asoleado,San Diego,CA,91901,8E - Tavern Rd (go South) - S Grade (go East) ...,None,Rancho Palo Verde
149111,2123,None,DELAND CT,San Diego,CA,91901,S Grade to Deland Dr to Deland Ct Cross Street...,Alpine,Alpine
133282,2123,None,DELAND CT,San Diego,CA,91901,S Grade to Deland Dr to Deland Ct,None,Alpine
55955,1915,None,Ranchito Ln,San Diego,CA,91901,None,None,Alpine
99892,1434,None,Marshall Rd,San Diego,CA,91901,Cross Street: Alpine Blvd.,Alpine,Alpine
44571,5702,None,Yearling Court,San Diego,CA,91902,None,None,Bonita Highlands


In [5]:
import tensorflow as tf
import numpy as np

In [6]:
df2 = df['streetnumber'].map(lambda s: '{!s:6.6}'.format(str(s))) + ' ' + df['streetname'].map(lambda s: '{!s:20.20}'.format(s)) + ' ' + df['city'] + ' ' + df['zip5']

In [7]:
df2[:8]

0    4660   Coconino Way         San Diego 92117
1    3485   Hawk St.             San Diego 92103
2    10464  Cheviot Ct.          San Diego 92126
3    3814   Merivale Ave         San Diego 92116
4    1212   Seagreen Pl          San Diego 92154
5    1706   Malden               San Diego 92109
6    8891   Armorss Avenue       San Diego 92123
7    2909   Keats St             San Diego 92106
dtype: object

In [8]:
df3 = df2.map(lambda s: list(map(ord, list(s))))

In [9]:
df3[:8]

0    [52, 54, 54, 48, 32, 32, 32, 67, 111, 99, 111,...
1    [51, 52, 56, 53, 32, 32, 32, 72, 97, 119, 107,...
2    [49, 48, 52, 54, 52, 32, 32, 67, 104, 101, 118...
3    [51, 56, 49, 52, 32, 32, 32, 77, 101, 114, 105...
4    [49, 50, 49, 50, 32, 32, 32, 83, 101, 97, 103,...
5    [49, 55, 48, 54, 32, 32, 32, 77, 97, 108, 100,...
6    [56, 56, 57, 49, 32, 32, 32, 65, 114, 109, 111...
7    [50, 57, 48, 57, 32, 32, 32, 75, 101, 97, 116,...
dtype: object

In [10]:
N_CODES = 128
def maprow(row):
    sess = tf.Session()
    v = sess.run(tf.one_hot(row, N_CODES))
    return [flatten for sub in v for flatten in sub]
    

In [11]:
train_inputs = df3[:10].map(maprow)
train_inputs

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
dtype: object

In [12]:
train_labels = train_inputs
len(train_inputs[0])

5504

In [13]:
X = tf.placeholder(tf.float32, shape=[1, len(train_inputs[0])])
Y = tf.placeholder(tf.float32, shape=[1, len(train_labels[0])])

In [14]:
HIDDEN=10
parameters = {
		'W1': tf.Variable(tf.random_normal([len(train_inputs[0]), HIDDEN])),
		'b1': tf.Variable(tf.random_normal([HIDDEN])),
		'W2': tf.Variable(tf.random_normal([HIDDEN, len(train_labels[0])])),
		'b2': tf.Variable(tf.random_normal([len(train_labels[0])]))
}

In [15]:
num_epochs = 2000
learning_rate = 0.0005

In [16]:
def neural_net(X,parameters):
	Z1 = tf.add(tf.matmul(X, parameters['W1']), parameters['b1'])
	A2 = tf.nn.relu(Z1)
	Z2 = tf.add(tf.matmul(A2, parameters['W2']), parameters['b2'])
	return Z2

In [17]:
def train():
    SAMPLES = len(train_inputs)
    INPUT_VAR_CODES = len(train_inputs[0])
    INPUT_VARS = INPUT_VAR_CODES // N_CODES
    OUTPUT_VAR_CODES = len(train_labels[0])
    OUTPUT_VARS = OUTPUT_VAR_CODES // N_CODES
    Z = neural_net(X,parameters)
    costs = []
    optimizers = []
    for i in range(OUTPUT_VARS):
        c = Z[0][N_CODES*i:N_CODES*i+N_CODES]
        costs.append(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=c,  labels=Y[0][N_CODES*i:N_CODES*i+N_CODES])))
        optimizer_k = optimizers.append(tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(costs[i]))
    optimizer = tf.group(*optimizers)
    tf.Print('costs:',costs)
    cost = tf.reduce_sum(costs)
    print(costs)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)
        epoch = 0
        while epoch<num_epochs:
            for i in range(SAMPLES):
                _ , c = sess.run([optimizer, cost], feed_dict={
                    X: np.reshape(train_inputs[i],[1,INPUT_VAR_CODES]), 
                    Y: np.reshape(train_labels[i],[1,OUTPUT_VAR_CODES])
                }) 
                if c <= 0.000001:
                    epoch = num_epochs
                    break
            if epoch % 200 == 0 or epoch == num_epochs:
                print ("Cost after epoch %i: %f" % (epoch, c))
            epoch += 1
        saver.save(sess, 'model.ckpt')


In [18]:
train()

W0930 01:08:06.617114 140460271224640 deprecation.py:323] From <ipython-input-17-e0201f704097>:12: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0930 01:08:12.580577 140460271224640 deprecation.py:323] From <ipython-input-17-e0201f704097>:15: Print (from tensorflow.python.ops.logging_ops) is deprecated and will be removed after 2018-08-20.
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf

[<tf.Tensor 'Mean:0' shape=() dtype=float32>, <tf.Tensor 'Mean_1:0' shape=() dtype=float32>, <tf.Tensor 'Mean_2:0' shape=() dtype=float32>, <tf.Tensor 'Mean_3:0' shape=() dtype=float32>, <tf.Tensor 'Mean_4:0' shape=() dtype=float32>, <tf.Tensor 'Mean_5:0' shape=() dtype=float32>, <tf.Tensor 'Mean_6:0' shape=() dtype=float32>, <tf.Tensor 'Mean_7:0' shape=() dtype=float32>, <tf.Tensor 'Mean_8:0' shape=() dtype=float32>, <tf.Tensor 'Mean_9:0' shape=() dtype=float32>, <tf.Tensor 'Mean_10:0' shape=() dtype=float32>, <tf.Tensor 'Mean_11:0' shape=() dtype=float32>, <tf.Tensor 'Mean_12:0' shape=() dtype=float32>, <tf.Tensor 'Mean_13:0' shape=() dtype=float32>, <tf.Tensor 'Mean_14:0' shape=() dtype=float32>, <tf.Tensor 'Mean_15:0' shape=() dtype=float32>, <tf.Tensor 'Mean_16:0' shape=() dtype=float32>, <tf.Tensor 'Mean_17:0' shape=() dtype=float32>, <tf.Tensor 'Mean_18:0' shape=() dtype=float32>, <tf.Tensor 'Mean_19:0' shape=() dtype=float32>, <tf.Tensor 'Mean_20:0' shape=() dtype=float32>, <tf

In [19]:
def test():
    # Test predictions by computing the output using training set as input
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        saver = tf.train.import_meta_graph('model.ckpt.meta')
        saver.restore(sess,'model.ckpt')
        for row in range(len(train_inputs)):

            g = train_inputs[row]
            g = np.reshape(g,[1,INPUT_VAR_CODES])
            output = neural_net(g,parameters)
            outputs = []
            for i in range(OUTPUT_VARS):
                kk = tf.nn.softmax(output[0][N_CODES*i : N_CODES*i+N_CODES])
                outputs.append(kk)
                
            
            out = sess.run(outputs)
            out = np.reshape(list(map(list,out)), [1, OUTPUT_VAR_CODES])[0].tolist()
            out = list(map(lambda x: float("%.1f" % x), out))
            print("\nROW #" + str(row))
            print("Expected: " + str(train_labels[row]))
            row = row + 1
            print("Actual..: " + str(out))

In [20]:
test()

W0930 01:11:50.912294 140460271224640 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1282: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


NameError: name 'INPUT_VAR_CODES' is not defined